In [83]:
import ssl
import pandas as pd
import numpy as np
import requests
import certifi
import osmnx as ox
import networkx as nx
import folium
from geopy.geocoders import Nominatim
from concurrent.futures import ProcessPoolExecutor

pd.set_option('display.max_columns', None)  # This will show all columns
pd.set_option('display.width', None) 

In [84]:


# -------------------------
# Setup: SSL and Geolocator
# -------------------------
ssl_context = ssl.create_default_context(cafile=certifi.where())
geolocator = Nominatim(user_agent='wichita_nav', ssl_context=ssl_context)

# ----------------------------------------
# Download and preprocess road network data
# ----------------------------------------
G_multi = ox.graph_from_place('Wichita, Kansas, USA', network_type='drive')
ox.add_edge_speeds(G_multi)
ox.add_edge_travel_times(G_multi)

# Convert MultiGraph to DiGraph, keeping only the fastest edge between nodes
def multigraph_to_digraph(G_multi):
    G_simple = nx.DiGraph()
    # Copy nodes with attributes
    G_simple.add_nodes_from(G_multi.nodes(data=True))
    # For each edge, keep the one with the least travel_time
    for u, v, data in G_multi.edges(data=True):
        if G_simple.has_edge(u, v):
            if data.get('travel_time', float('inf')) < G_simple[u][v].get('travel_time', float('inf')):
                G_simple[u][v].update(data)
        else:
            G_simple.add_edge(u, v, **data)
    return G_simple

G = multigraph_to_digraph(G_multi)
G.graph["crs"] = G_multi.graph.get("crs")

# -------------------------------
# Helper functions for visualization
# -------------------------------
def plot_route(G, route, route_map, color='blue'):
    route_points = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
    folium.PolyLine(route_points, color=color, weight=4.5, opacity=0.7).add_to(route_map)

def get_nearest_node(address): 
    location = geolocator.geocode(address, timeout=5)
    if location:
        print(f"Geocoded Address: {address} -> ({location.latitude}, {location.longitude})")
        return ox.distance.nearest_nodes(G, X=location.longitude, Y=location.latitude)
    else: 
        print(f"Failed to geocode address: {address}")
        return None

def get_travel_time(G, route): 
    travel_time = 0
    for u, v in zip(route[:-1], route[1:]):
        travel_time += G[u][v]['travel_time']
    return travel_time

# Highway score: proportion of edges that are main roads
def highway_score(G, route): 
    main_road_types = ['motorway', 'trunk', 'primary', 'secondary']
    edges = list(zip(route, route[1:]))
    count = 0
    for u, v in edges: 
        edge_data = G.get_edge_data(u, v)
        if edge_data: 
            highway = edge_data.get('highway', None)
            if isinstance(highway, list):
                if any(h in main_road_types for h in highway):
                    count += 1
            else:
                if highway in main_road_types:
                    count += 1
    return count / len(edges) if edges else 0

# -------------------------------
# Define custom weight functions
# -------------------------------
# 1. Fastest route: use existing travel_time weight.
# 2. Highway Priority: add a penalty for non-highway segments.
def highway_priority_weight(u, v, data):
    main_road_types = ['motorway', 'trunk', 'primary', 'secondary']
    highway = data.get('highway', None)
    if isinstance(highway, list):
        is_highway = any(h in main_road_types for h in highway)
    else:
        is_highway = highway in main_road_types
    penalty = 0 if is_highway else 60  # add a 60-second penalty for non-highway segments
    return data['travel_time'] + penalty

# 3. Balanced Route: give a bonus for highway segments (subtract time) but avoid negative weights.
def balanced_route_weight(u, v, data):
    main_road_types = ['motorway', 'trunk', 'primary', 'secondary']
    highway = data.get('highway', None)
    if isinstance(highway, list):
        is_highway = any(h in main_road_types for h in highway)
    else:
        is_highway = highway in main_road_types
    bonus = 30 if is_highway else 0  # subtract 30 seconds if on a highway
    return max(data['travel_time'] - bonus, 1)  # ensure cost is at least 1 second

# -------------------------------
# Define start and end points
# -------------------------------
start_address = "7331 Ayesbury Cir, Wichita KS"
# end_address = "1220 N Tyler Rd, Wichita, KS"
end_address = "123 Seneca St, Wichita, KS"

start_node = get_nearest_node(start_address)
end_node = get_nearest_node(end_address)

# -------------------------------
# Compute routes using the three algorithms
# -------------------------------
# Fastest Route (using travel_time)
fastest_route = nx.shortest_path(G, start_node, end_node, weight='travel_time')

# Highway Priority Route (penalize non-highway segments)
highway_route = nx.shortest_path(G, start_node, end_node, weight=highway_priority_weight)

# Balanced Route (bonus for highways)
balanced_route = nx.shortest_path(G, start_node, end_node, weight=balanced_route_weight)

# -------------------------------
# Collect routes and compute metrics
# -------------------------------
routes = [
    {"name": "Fastest", "route": fastest_route, "color": "blue"},
    {"name": "Highway Priority", "route": highway_route, "color": "red"},
    {"name": "Balanced", "route": balanced_route, "color": "green"}
]

for r in routes:
    t = get_travel_time(G, r["route"])
    h = highway_score(G, r["route"])
    print(f"{r['name']} Route: Travel Time = {t:.1f} sec, Highway Score = {h:.2f}")

# -------------------------------
# Visualize routes on Folium map
# -------------------------------
location_wichita = geolocator.geocode("Wichita, Kansas, USA")
route_map = folium.Map(location=[location_wichita.latitude, location_wichita.longitude], zoom_start=12)

for r in routes:
    plot_route(G, r["route"], route_map, color=r["color"])

# Save the three routes with their nodes
fastest_route_nodes = fastest_route
highway_route_nodes = highway_route
balanced_route_nodes = balanced_route

# Display the interactive map (in Jupyter, the map will render in the output cell)
route_map


Geocoded Address: 7331 Ayesbury Cir, Wichita KS -> (37.727137306122444, -97.25127428571429)
Geocoded Address: 123 Seneca St, Wichita, KS -> (37.68368036734694, -97.35312542857143)
Fastest Route: Travel Time = 779.3 sec, Highway Score = 0.79
Highway Priority Route: Travel Time = 828.6 sec, Highway Score = 0.93
Balanced Route: Travel Time = 910.9 sec, Highway Score = 0.91


In [85]:
# Get the list of nodes that each route goes through
for r in routes:
    print(f"{r['name']} Route Nodes: {r['route']}")

Fastest Route Nodes: [122229824, 122284236, 122284222, 122344313, 122459139, 674282359, 122438959, 122221221, 122459110, 122493958, 7841916281, 12392675685, 12392675686, 8251743478, 122493956, 122188923, 122189049, 122493954, 122232404, 122319979, 122186893, 122493952, 1469981594, 122186786, 122247329, 122476199, 122344294, 122493946, 122162432, 122226134, 9874997853, 9874997854, 9874997851, 9875008241, 9875008240, 122226126, 122226118, 122226115, 122226111, 122226109, 122226106, 122216596, 122226105, 11328745190, 10867456876, 122226097, 122226091, 122226086, 122226081, 122187555, 122213606, 122226075, 122194757, 122226067, 122226064, 122226061, 122223089, 122223093, 122223098, 122223101, 122223104, 122223111, 122223120, 122497137, 122134912, 122134878, 122152859, 122152028, 122168521, 122149823, 122132448, 122146912, 122153669, 498268187, 122136801, 122133526, 122136762, 122543576, 122486999, 122543573, 122439395, 122472427]
Highway Priority Route Nodes: [122229824, 122284236, 1222842

In [86]:
factors = pd.read_csv('./crash_factors.csv')
factors.head()

,X,Y,OBJECTID,INCIDENTID,DATE,INTERSECTION,LOCATION,CROSS_STREET,CITY,STATE,POSTAL_CODE,OFFENSE_CO,OFFENSE_DE,LONGITUDE,LATITUDE,YEAR_,MONTH_,DOW_,MONTHFULL,DOWFULL,QUARTER,STATION,NAME,PRCP,SNOW,SNWD
0,-1.580702e+07,4.523771e+06,1,23C013957,2024-02-29,W 21ST ST N & N WACO AVE,W 21ST ST N,N WACO AVE,WICHITA,KS,67203.0,7090,MOTOR VEHICLE/FIXED OBJECT,-141.996855,37.604971,2024,2,4,February,Thursday,1,US1KSSG0124USC00148847US1KSSG0010US1KSSG0052US...,"WICHITA 6.3 W, KS USWICHITA WX, KS USWICHITA 3...",0.0,0.0,0.0
1,-1.580511e+07,4.521643e+06,2,24C000041,2024-01-01,1145 E LINCOLN ST,1145 E LINCOLN ST,0,WICHITA,KS,67211.0,7030,MOTOR VEHICLE/MOTOR VEHICLE IN TRANSPORT,-141.979709,37.589825,2024,1,1,January,Monday,1,US1KSSG0124USC00148847US1KSSG0010US1KSSG0052US...,"WICHITA 6.3 W, KS USWICHITA WX, KS USWICHITA 3...",0.0,0.0,0.0
2,-1.580513e+07,4.521632e+06,3,24C000041,2024-01-01,E LINCOLN ST & S IDA AVE,E LINCOLN ST,S IDA AVE,WICHITA,KS,67211.0,7030,MOTOR VEHICLE/MOTOR VEHICLE IN TRANSPORT,-141.979907,37.589743,2024,1,1,January,Monday,1,US1KSSG0124USC00148847US1KSSG0010US1KSSG0052US...,"WICHITA 6.3 W, KS USWICHITA WX, KS USWICHITA 3...",0.0,0.0,0.0
3,-1.580504e+07,4.523029e+06,4,24C000084,2024-01-01,2330 E CENTRAL AVE,2330 E CENTRAL AVE,0,WICHITA,KS,67214.0,6105,PRIVATE PROPERTY/NON-INJURY CRASH,-141.979034,37.599688,2024,1,1,January,Monday,1,US1KSSG0124USC00148847US1KSSG0010US1KSSG0052US...,"WICHITA 6.3 W, KS USWICHITA WX, KS USWICHITA 3...",0.0,0.0,0.0
4,-1.580542e+07,4.521992e+06,5,24C000127,2024-01-01,951 E DEWEY AVE,951 E DEWEY AVE,0,WICHITA,KS,67202.0,6105,PRIVATE PROPERTY/NON-INJURY CRASH,-141.982486,37.592306,2024,1,1,January,Monday,1,US1KSSG0124USC00148847US1KSSG0010US1KSSG0052US...,"WICHITA 6.3 W, KS USWICHITA WX, KS USWICHITA 3...",0.0,0.0,0.0


In [87]:
min_lat, max_lat = 37.5, 37.8
min_lon, max_lon = -97.5, -97.0

# Filter factors to points within Wichita area
factors = factors[
    (factors['LATITUDE'] >= min_lat) & (factors['LATITUDE'] <= max_lat) &
    (factors['LONGITUDE'] >= min_lon) & (factors['LONGITUDE'] <= max_lon)
]

factors[["LATITUDE", "LONGITUDE"]].describe()

,LATITUDE,LONGITUDE
count,9523.000000,9523.000000
mean,37.687606,-97.330206
std,0.032844,0.067050
min,37.522456,-97.499607
25%,37.669723,-97.370793
50%,37.686072,-97.331388
75%,37.708795,-97.280555
max,37.796193,-97.153047


In [88]:
factors["LONGITUDE"] = factors["LONGITUDE"].astype(float)
factors["LATITUDE"] = factors["LATITUDE"].astype(float)

# Reproject the graph to EPSG:4326 (if not already in that CRS)
Graph = ox.project_graph(G_multi, to_crs='EPSG:4326')   

if not Graph.graph['crs'].equals('EPSG:4326'):
    print("Warning: Graph CRS is not EPSG:4326.")


# batch find nearest nodes
nearest_nodes = ox.distance.nearest_nodes(Graph, X=x, Y=y)

# assign results to new column
factors["NEAREST_NODE"] = nearest_nodes

# factors.head(50)


print(f"{factors['NEAREST_NODE'].nunique()} out of {factors['NEAREST_NODE'].count()}")

factors.head(5)


3399 out of 9523


,X,Y,OBJECTID,INCIDENTID,DATE,INTERSECTION,LOCATION,CROSS_STREET,CITY,STATE,POSTAL_CODE,OFFENSE_CO,OFFENSE_DE,LONGITUDE,LATITUDE,YEAR_,MONTH_,DOW_,MONTHFULL,DOWFULL,QUARTER,STATION,NAME,PRCP,SNOW,SNWD,NEAREST_NODE
312,-1.084954e+07,4.540834e+06,313,24C007787,2024-01-15,2441 N MAIZE RD,2441 N MAIZE RD,0,WICHITA,KS,67205.0,6105,PRIVATE PROPERTY/NON-INJURY CRASH,-97.463081,37.726301,2024,1,1,January,Monday,1,US1KSSG0124USC00148847US1KSSG0010US1KSSG0052US...,"WICHITA 6.3 W, KS USWICHITA WX, KS USWICHITA 3...",0.00,0.0,0.0,122218038
469,-1.083338e+07,4.537202e+06,470,24C011789,2024-01-12,E 8TH ST N & E 9TH ST N,E 8TH ST N,N I135 HWY,WICHITA,KS,67214.0,7030,MOTOR VEHICLE/MOTOR VEHICLE IN TRANSPORT,-97.317908,37.700493,2024,1,5,January,Friday,1,US1KSSG0124USC00148847US1KSSG0010US1KSSG0052US...,"WICHITA 6.3 W, KS USWICHITA WX, KS USWICHITA 3...",0.00,0.0,0.0,660386774
604,-1.083074e+07,4.531584e+06,605,24C014656,2024-01-27,S CLIFTON AVE & E FUNSTON ST,S CLIFTON AVE,E FUNSTON ST,WICHITA,KS,67218.0,7050,MOTOR VEHICLE/PARKED,-97.294217,37.660551,2024,1,6,January,Saturday,1,US1KSSG0124US1KSSG0179USC00148847US1KSSG0010US...,"WICHITA 6.3 W, KS USWICHITA 3.2 SE, KS USWICHI...",0.05,0.0,0.0,122363017
678,-1.082803e+07,4.532429e+06,679,24C016429,2024-01-30,1200 S CHRISTINE RD,1200 S CHRISTINE RD,0,WICHITA,KS,67218.0,6155,NON-INJURY CRASH/HIT AND RUN/UNDER $1000,-97.269850,37.666559,2024,1,2,January,Tuesday,1,US1KSSG0124US1KSSG0179USC00148847US1KSSG0010US...,"WICHITA 6.3 W, KS USWICHITA 3.2 SE, KS USWICHI...",0.00,0.0,0.0,122316876
1021,-1.083250e+07,4.530097e+06,1022,24C023781,2024-02-10,E PAWNEE AVE & I135 HWY,E PAWNEE AVE,S 135 HWY,WICHITA,KS,67211.0,7030,MOTOR VEHICLE/MOTOR VEHICLE IN TRANSPORT,-97.309995,37.649973,2024,2,6,February,Saturday,1,US1KSSG0124US1KSSG0179USC00148847US1KSSG0010US...,"WICHITA 6.3 W, KS USWICHITA 3.2 SE, KS USWICHI...",0.00,0.0,0.0,445117693


In [89]:
route_one = factors["NEAREST_NODE"].isin(fastest_route_nodes)
route_two = factors["NEAREST_NODE"].isin(highway_route_nodes)
route_three = factors["NEAREST_NODE"].isin(balanced_route_nodes)

# Count the total number of crashes for each route.
crashes_route_one = factors[route_one].shape[0]
crashes_route_two = factors[route_two].shape[0]
crashes_route_three = factors[route_three].shape[0]

#Calculate the average crashes per node for each route
average_crashes_route_one = crashes_route_one / len(fastest_route_nodes) if fastest_route_nodes else 0
average_crashes_route_two = crashes_route_one / len(highway_route_nodes) if highway_route_nodes else 0
average_crashes_route_three = crashes_route_one / len(balanced_route_nodes) if balanced_route_nodes else 0

# Output the results.
print("Fastest Route:")
print(f"  Total crashes: {crashes_route_one}")
print(f"  Average crashes per node: {average_crashes_route_one:.2f}\n")

print("Highway Route:")
print(f"  Total crashes: {crashes_route_two}")
print(f"  Average crashes per node: {average_crashes_route_two:.2f}\n")

print("Balanced Route:")
print(f"  Total crashes: {crashes_route_three}")
print(f"  Average crashes per node: {average_crashes_route_three:.2f}")


Fastest Route:
  Total crashes: 291
  Average crashes per node: 3.55

Highway Route:
  Total crashes: 308
  Average crashes per node: 2.33

Balanced Route:
  Total crashes: 312
  Average crashes per node: 2.51


Weather Data

In [90]:
#Get grid cordinates needed for the forcast

# Wichita coordinates
lat = 37.6872
lon = -97.3301

# NWS API endpoint for grid points
points_url = f"https://api.weather.gov/points/{lat},{lon}"

#Fetch data
response = requests.get(points_url)
response.raise_for_status() #Raise error if occurs
grid_data = response.json()

#Extract grid ID & x, y Cordinates
grid_id = grid_data['properties']['gridId']
grid_x = grid_data['properties']['gridX']
grid_y = grid_data['properties']['gridY']

print(f"Grid ID: {grid_id}, Grid X: {grid_x}, Grid Y: {grid_y}")


Grid ID: ICT, Grid X: 62, Grid Y: 34


In [91]:
#Get forcast

# NWS API endpoint for forecast
forecast_url = f"https://api.weather.gov/gridpoints/{grid_id}/{grid_x},{grid_y}/forecast"

#Fetch forcast data
response = requests.get(forecast_url)
response.raise_for_status() #Check if error
forecast_data = response.json()

#Get and display current forcast
current_forecast = forecast_data['properties']['periods'][0]
print(f"Current conditions: {current_forecast['shortForecast']}, Temperature: {current_forecast['temperature']}°{current_forecast['temperatureUnit']}")



Current conditions: Sunny, Temperature: 73°F


In [92]:
# Check current weather conditions (rain or snow)
current_forecast = forecast_data['properties']['periods'][0]
current_condition = current_forecast['shortForecast'].lower()

# Determine if it's raining or snowing
is_raining_or_snowing = ('rain' in current_condition or 'snow' in current_condition)

is_raining_or_snowing

False

Check for bad weather on 3 routes

In [93]:
# Create the bad weather condition based on PRCP > 0 or SNOW > 0
bad_weather = (factors["PRCP"] > 0).astype(bool) | (factors["SNOW"] > 0).astype(bool)

#Count crashes on each route during bad weather
crashes_route_one_bad_weather = factors[route_one & bad_weather].shape[0]
crashes_route_two_bad_weather = factors[route_two & bad_weather].shape[0]
crashes_route_three_bad_weather = factors[route_three & bad_weather].shape[0]

#Calculate the average number of crashes per node during bad weather
average_crashes_route_one_bad_weather = crashes_route_one_bad_weather / len(fastest_route_nodes) if fastest_route_nodes else 0
average_crashes_route_two_bad_weather = crashes_route_two_bad_weather / len(highway_route_nodes) if highway_route_nodes else 0
average_crashes_route_three_bad_weather = crashes_route_three_bad_weather / len(balanced_route_nodes) if balanced_route_nodes else 0

# Output the results for bad weather
print("Fastest Route (Bad Weather):")
print(f"  Total crashes: {crashes_route_one_bad_weather} with weather")
print(f"  Average crashes per node: {average_crashes_route_one_bad_weather:.2f}\n")

print("Highway Route (Bad Weather):")
print(f"  Total crashes: {crashes_route_two_bad_weather} with weather")
print(f"  Average crashes per node: {average_crashes_route_two_bad_weather:.2f}\n")

print("Balanced Route (Bad Weather):")
print(f"  Total crashes: {crashes_route_three_bad_weather} with weather")
print(f"  Average crashes per node: {average_crashes_route_three_bad_weather:.2f}")

Fastest Route (Bad Weather):
  Total crashes: 119 with weather
  Average crashes per node: 1.45

Highway Route (Bad Weather):
  Total crashes: 138 with weather
  Average crashes per node: 1.10

Balanced Route (Bad Weather):
  Total crashes: 139 with weather
  Average crashes per node: 1.20


Risk score = a (Number of Crashes) + b (Number of rain / snow crashes) (Current weather)

a & b are parameters to be learned
if current weather has rain/snow - current weather = 1, else current weather = 0

In [97]:
def calculate_risk_score (avg_crashes, avg_weather_crashes, weather:bool):
    if weather:
        # Boost the risk score with rain/snow crashes when current weather is bad.
        return avg_crashes + (2 * avg_weather_crashes)
    else: 
        return avg_crashes


route_one_risk = calculate_risk_score(average_crashes_route_one, average_crashes_route_one_bad_weather, is_raining_or_snowing)
route_two_risk = calculate_risk_score(average_crashes_route_two, average_crashes_route_two_bad_weather, is_raining_or_snowing)
route_three_risk = calculate_risk_score(average_crashes_route_three, average_crashes_route_three_bad_weather, is_raining_or_snowing)

# print(f"The risk score for route 1 is {route_one_risk}")
# print(f"The risk score for route 2 is {route_two_risk}")
# print(f"The risk score for route 3 is {route_three_risk}")

routes = [
    {"name": "Fastest", "route": fastest_route, "color": "blue", "risk":route_one_risk},
    {"name": "Highway Priority", "route": highway_route, "color": "red", "risk":route_two_risk},
    {"name": "Balanced", "route": balanced_route, "color": "green", "risk":route_three_risk}
]

for r in routes:
    t = get_travel_time(G, r["route"])
    h = highway_score(G, r["route"])
    risk = r["risk"]
    
    # Convert seconds to minutes and seconds
    minutes = t // 60
    seconds = t % 60

    print(f"{r['name']} Route: Travel Time = {int(minutes)} min {seconds:.1f} sec, Highway Score = {h:.2f}, Risk Score: {risk:.2f}")


Fastest Route: Travel Time = 12 min 59.3 sec, Highway Score = 0.79, Risk Score: 3.55
Highway Priority Route: Travel Time = 13 min 48.6 sec, Highway Score = 0.93, Risk Score: 2.33
Balanced Route: Travel Time = 15 min 10.9 sec, Highway Score = 0.91, Risk Score: 2.51
